In [12]:
import pandas as pd
import openpyxl
import os


directory_path = "C:\\Users\\fdona\\OneDrive\\Escritorio\\Compilado_Mpl_FD\\SUBTERRRANEAS_OBRAS_LINEALES"


data_total = pd.DataFrame()


excel_files = [file for file in os.listdir(directory_path) if file.endswith(('.xlsx'))]

# Procesar cada archivo Excel encontrado
for file in excel_files:
    file_path = os.path.join(directory_path, file)  # Construir la ruta completa al archivo
    xls = pd.ExcelFile(file_path)
    wb = openpyxl.load_workbook(file_path)

    for hoja_archivo in xls.sheet_names:
        hoja = wb[hoja_archivo]
        rango = hoja['C133:AZ133']

        ultima_celda_con_datos = None
        for celda in rango[0]:
            if celda.value is not None:
                ultima_celda_con_datos = celda

        if ultima_celda_con_datos:
            columna = ''.join(filter(str.isalpha, ultima_celda_con_datos.coordinate))
            print(f'La última columna con datos es {columna}, en la hoja: {hoja_archivo}')
    
    
            data_identificacion = pd.read_excel(xls, sheet_name= hoja_archivo, skiprows=132, nrows=8, usecols=f"C:{columna}")


            data_identificacion_transposed = data_identificacion.transpose()


            data_identificacion_transposed.columns = data_identificacion_transposed.iloc[0]
            data_identificacion_transposed = data_identificacion_transposed.drop(data_identificacion_transposed.index[0])
            # nombramiento de columnmas
            data_identificacion_transposed.columns = ['Fecha', 'Id_Certificado', 'Comentarios', 'Año_QAQC', 'Fecha_QAQC', 'Mes_QAQC', 'Muestras_Con_Datos', 'Muestras_Totales']
            data_identificacion_transposed['Index_Column'] = data_identificacion_transposed.index

            # Reiniciar el índice para crear un nuevo índice  AUTOincremental
            data_identificacion_transposed.reset_index(drop=True, inplace=True)
            data_identificacion_transposed.columns = ['Fecha', 'Id_Certificado', 'Comentarios', 'Año_QAQC', 'Fecha_QAQC', 'Mes_QAQC', 'Muestras_Con_Datos', 'Muestras_Totales', 'Estacion']
            # ORDEN DE LA COLUMNAS 
            data_identificacion_transposed = data_identificacion_transposed[['Estacion','Fecha', 'Id_Certificado', 'Comentarios', 'Año_QAQC', 'Fecha_QAQC', 'Mes_QAQC', 'Muestras_Con_Datos', 'Muestras_Totales']]
            # data_identificacion total
            data_total = pd.concat([data_total, data_identificacion_transposed], ignore_index=True)
            
        else:
            print(f'No hay datos en el rango C133:AZ133, para la hoja: {hoja_archivo}')



La última columna con datos es O, en la hoja:  2007
La última columna con datos es O, en la hoja: 2008
La última columna con datos es O, en la hoja: 2009
La última columna con datos es Y, en la hoja: 2010(1semestre)
La última columna con datos es Y, en la hoja: 2010 (2 semestre)
La última columna con datos es AU, en la hoja: 2011
La última columna con datos es AU, en la hoja: 2012
La última columna con datos es AB, en la hoja: 2013
La última columna con datos es Z, en la hoja: 2014
La última columna con datos es AA, en la hoja: 2015
La última columna con datos es AA, en la hoja: 2016
La última columna con datos es AA, en la hoja: 2017
La última columna con datos es V, en la hoja: 2018
La última columna con datos es AA, en la hoja: 2019
La última columna con datos es U, en la hoja: 2020
La última columna con datos es U, en la hoja: 2021
La última columna con datos es O, en la hoja: 2022
La última columna con datos es AA, en la hoja: 2023
No hay datos en el rango C133:AZ133, para la hoja

In [14]:
data_total

,Estacion,Fecha,Id_Certificado,Comentarios,Año_QAQC,Fecha_QAQC,Mes_QAQC,Muestras_Con_Datos,Muestras_Totales
0,PMA-11,00:00:00,*,NaN,2007,00-01-1900,Enero,6,12
1,PMA-11.1,00:00:00,*,NaN,2007,00-01-1900,Febrero,6,12
2,PMA-11.2,00:00:00,*,NaN,2007,00-01-1900,Marzo,6,12
3,PMA-11.3,00:00:00,*,NaN,2007,00-01-1900,Abril,6,12
4,PMA-11.4,2007-05-25 00:00:00,704765A,NaN,2007,25-05-2007,Mayo,6,12
...,...,...,...,...,...,...,...,...,...
501,Sondaje AP Los Vilos.7,2023-08-08 00:00:00,2023-289658,NaN,2023,08-08-2023,Agosto,0,12
502,Sondaje AP Los Vilos.8,2023-09-05 00:00:00,2023-296127,NaN,2023,05-09-2023,Septiembre,0,12
503,Sondaje AP Los Vilos.9,2023-10-03 00:00:00,2023-311612,NaN,2023,03-10-2023,Octubre,0,12
504,Sondaje AP Los Vilos.10,2023-11-14 00:00:00,2023-331065,NaN,2023,14-11-2023,Noviembre,0,12
